In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

# Notebook to summarise uses of ExternalGalaxyAstrometry package

In [ ]:
import ExternalGalaxyAstrometry as EGA

Input file is (by default) drawn from the Gaia archive. In this example we draw it from a 8 degree radius around the photometric centre of the LMC

For simplicity while this software is testing this is saved into a local file

In [ ]:
input_name ='testfile.csv'

dataLMC = pd.read_csv(input_name)

In [ ]:
dataLMC

In [ ]:
import importlib
importlib.reload(EGA)

In [ ]:
dataLMCfilt = EGA.filterUsingProperMotions(dataLMC, a0=EGA.a0GaiaEDR3, d0=EGA.d0GaiaEDR3 ,verbose=True)

## Compare distributions on sky of the two

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(12,6))
ax[0].hist2d(dataLMCfilt['ra'],dataLMCfilt['dec'],bins=100)
ax[1].hist2d(dataLMC['ra'],dataLMC['dec'], bins=100)
plt.show()

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(12,6))

for ax, df, label in zip(ax, [dataLMCfilt, dataLMC], ['filtered', 'unfiltered']):
    ax.hist2d(df['phot_bp_mean_mag']-df['phot_rp_mean_mag'],
         df['phot_g_mean_mag'],[np.linspace(-1,4,100),np.linspace(10,20,100)],cmin=1,
          norm=mpl.colors.LogNorm(),cmap='Greys')
    ymin,ymax = ax.get_ylim()
    ax.set_ylim(ymax,ymin)
    ax.set_title(label)
plt.show()

In [ ]:
binneddata = EGA.binDataOnSky(dataLMCfilt, a0=EGA.a0GaiaEDR3, d0=EGA.d0GaiaEDR3, statisticBins=10, verbose=True)

In [ ]:
binneddata.columns

In [ ]:
plt.scatter(binneddata['x'],binneddata['y'],c=binneddata['muxBinned'],cmap='RdBu')
plt.colorbar()
plt.show()

In [ ]:
import importlib
importlib.reload(EGA)

In [ ]:
binneddata_filt = binneddata[binneddata['countBinned']>10]
results, _ = EGA.fitRotationCurveModel(binneddata_filt, fixedParams={'a0':EGA.a0GaiaEDR3,'d0':EGA.d0GaiaEDR3, }, 
                          guessParams={'mu_x0':np.nanmedian(binneddata['muxBinned']),'mu_y0':np.nanmedian(binneddata['muyBinned']),
                                    'mu_z0':1.1,'i':30,'Omega':300,'v0':0.3,'R0':0.1,'alpha_RC':1.5})
results